In [8]:
import os
import pandas as pd

In [9]:
def get_data_file_path(current_trial_name="v2_eval_20_3", base_folder="game_logs", trial_base_name="alfworld_eval_", csv_file_name="alfworld_results.csv"):
    """returns the csv file path."""
    current_trial_folder = trial_base_name + current_trial_name
    file_path = os.path.join(base_folder, current_trial_folder, csv_file_name)
    return file_path

In [26]:
CURRENT_TRIAL_NAME = "v2_eval_20_3"
file_path = get_data_file_path(current_trial_name=CURRENT_TRIAL_NAME)
results_data = pd.read_csv(os.path.join("..",file_path))

In [27]:
header_order = ["model", "keys_removed","temperature","env_type","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions","error"]
clean_results = results_data[header_order]
clean_results.head()

,model,keys_removed,temperature,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,error
0,gpt-3.5-turbo-0125,react-1,0.0,cool,False,True,NaN,49,12,0,NaN
1,gpt-3.5-turbo-0125,react-1,0.0,clean,False,True,NaN,49,13,0,NaN
2,gpt-3.5-turbo-0125,react-1,0.0,clean,False,True,NaN,49,33,0,NaN
3,gpt-3.5-turbo-0125,react-1,0.0,put,False,True,NaN,49,26,0,NaN
4,gpt-3.5-turbo-0125,react-1,0.0,clean,False,True,NaN,49,39,2,NaN


In [28]:
clean_results.fillna(0,inplace=True )
clean_results.head()

/var/folders/y4/c4phbshn5739xq523fgf_cl40000gn/T/ipykernel_25805/1470857794.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_results.fillna(0,inplace=True )


,model,keys_removed,temperature,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,error
0,gpt-3.5-turbo-0125,react-1,0.0,cool,False,True,0,49,12,0,0.0
1,gpt-3.5-turbo-0125,react-1,0.0,clean,False,True,0,49,13,0,0.0
2,gpt-3.5-turbo-0125,react-1,0.0,clean,False,True,0,49,33,0,0.0
3,gpt-3.5-turbo-0125,react-1,0.0,put,False,True,0,49,26,0,0.0
4,gpt-3.5-turbo-0125,react-1,0.0,clean,False,True,0,49,39,2,0.0
